In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np

# Step 1: Load Dataset
# Replace 'IMDB Top 250 Movies.csv' with the actual dataset path
movies_df = pd.read_csv(r"C:\Users\anura\Downloads\RS DATASET\RS Practical 3/IMDB Top 250 Movies.csv")

# Select relevant columns and preprocess
movies_df = movies_df[['name', 'genre', 'tagline', 'casts', 'rating']].dropna()
movies_df['content'] = movies_df['genre'] + ' ' + movies_df['tagline'] + ' ' + movies_df['casts']
movies_df.dropna(inplace=True)
# Step 2: Content-Based Filtering
# Vectorize the 'content' column
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['content'])

# Compute cosine similarity for content-based filtering
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Step 3: Collaborative Filtering (using the 'rating' column)
# Create a user-item matrix based on movie ratings
ratings_data = movies_df[['name', 'rating']].copy()

# We assume each user has rated each movie, even though it’s implicit here.
# Create a user-item matrix for collaborative filtering
user_item_matrix = ratings_data.pivot(index='name', columns='rating', values='rating').fillna(0)
user_item_sparse = csr_matrix(user_item_matrix.values)

# Apply SVD for dimensionality reduction (Latent Factor Model)
svd = TruncatedSVD(n_components=5)  # Use 5 components for simplicity
latent_matrix = svd.fit_transform(user_item_sparse)

# Step 4: Define Hybrid Recommendation Functions

# Content-Based Recommendations
def get_content_based_recommendations(title, cosine_sim=cosine_sim):
    if title not in movies_df['name'].values:
        return ["Movie not found in database."]

    idx = movies_df.index[movies_df['name'] == title][0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    movie_indices = [i[0] for i in sim_scores[1:6]]  # Top 5 similar movies
    return movies_df['name'].iloc[movie_indices].tolist()

# Collaborative Recommendations
def get_collaborative_recommendations(user_id, user_item_matrix=user_item_matrix, latent_matrix=latent_matrix):
    if user_id not in user_item_matrix.index:
        return ["User not found in database."]

    user_idx = list(user_item_matrix.index).index(user_id)
    distances = pairwise_distances(latent_matrix[user_idx].reshape(1, -1), latent_matrix, metric='cosine').flatten()
    similar_users = distances.argsort()[1:4]  # Top 3 similar users

    recommended_movies = []
    for sim_user in similar_users:
        user_movies = user_item_matrix.iloc[sim_user].replace(0, None).dropna().index.tolist()
        recommended_movies.extend(user_movies)

    return list(set(recommended_movies))

# Hybrid Recommendations
# Refactor Hybrid Recommendations
def hybrid_recommendations(user_id, title, alpha=0.5):
    # Get Content-based recommendations
    content_recs = get_content_based_recommendations(title)

    # Handle collaborative recommendations with a simulated user
    collaborative_recs = get_collaborative_recommendations(user_id)

    # Combine recommendations
    # Using alpha to mix the two lists
    combined_recs = content_recs + collaborative_recs

    # Sort the final recommendations by relevance (you can modify this logic)
    final_recs = [(movie, alpha if movie in content_recs else 1-alpha) for movie in combined_recs]

    return sorted(final_recs, key=lambda x: x[1], reverse=True)

# Simulated User ID and Movie for testing
user_id = 1  # Simulate a user ID
movie_title = 'The Godfather'  # Movie title for which we want recommendations

# Print Hybrid Recommendations
print(f"Hybrid Recommendations for user {user_id} and movie '{movie_title}':")
print(hybrid_recommendations(user_id, movie_title))



Hybrid Recommendations for user 1 and movie 'The Godfather':
[('The Godfather Part II', 0.5), ('Scarface', 0.5), ('On the Waterfront', 0.5), ('Capernaum', 0.5), ('Heat', 0.5), ('User not found in database.', 0.5)]
